In [ ]:
%env MINERL_DATA_ROOT=/Volumes/CORSAIR/data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import minerl  # NOTE: we need gym>=0.13.1,<0.20
import gym
from train import get_agent, get_dynamics_environment, FMC, get_data_handler
from fgz.training.fgz_trainer import FGZTrainer
from fgz_config import FGZConfig
import torch
import wandb
from tqdm import tqdm

In [ ]:
torch.cuda.empty_cache()  # fix memory leaks

In [ ]:
enabled_tasks = [2, 3]  # cave and waterfall 
# enabled_tasks = [0, 1, 2, 3]  # all 

config = FGZConfig(
    enabled_tasks=enabled_tasks,
    disable_fmc_detection=True,  # if true, only classification will occur. 
    use_wandb=True,
    unroll_steps=64,
)

In [ ]:
minerl_env = gym.make('MineRLBasaltMakeWaterfall-v0')
agent = get_agent(config)
dynamics_env = get_dynamics_environment(config)

In [ ]:
agent.device

In [ ]:
# dynamics_env.batched_action_space_sample()

In [ ]:
# dummy_initial_state = torch.ones(4096, dtype=float)
# dynamics_env.set_all_states(dummy_initial_state)

In [ ]:
data_handler = get_data_handler(config, agent)
for loader in data_handler.loaders:
    print(loader)

In [ ]:
# t = data_handler.sample_single_trajectory()
# for window in t:
#     print(len(window))
#     for frame, state_embedding, action in window:
#         print(state_embedding.shape)
#         break
#     break

In [ ]:
dynamics_function_optimizer = torch.optim.Adam(
    dynamics_env.dynamics_function.parameters(),
    lr=0.001,
    # weight_decay=1e-4,
)

lr_scheduler = None
# lr_scheduler = torch.optim.lr_scheduler.StepLR(dynamics_function_optimizer, step_size=10, gamma=0.95)

In [ ]:
fmc = FMC(dynamics_env, freeze_best=True)
trainer = FGZTrainer(agent, fmc, data_handler, dynamics_function_optimizer, config=config)

if config.use_wandb:
    wandb.init(project="fgz-v0.1.1", config=config.asdict())

In [ ]:
# trainer.save("test_trainer_save.pth")
# loaded_trainer = FGZTrainer.load("test_trainer_save.pth", agent)
# loaded_trainer.evaluate("MineRLBasaltMakeWaterfall-v0", render=True, max_steps=4096, force_no_escape=True)

In [ ]:
# trainer.evaluate("MineRLBasaltMakeWaterfall-v0", render=True, max_steps=4096, force_no_escape=True)

In [ ]:
# trainer.eval_actions

In [ ]:
train_steps = 100
batch_size = 8
# train_steps = 5
checkpoint_every = 10
for train_step in tqdm(range(train_steps), desc="Training"):
    trainer.train_sub_trajectories(batch_size=batch_size, use_tqdm=False)
    if train_step % checkpoint_every == 0:
        trainer.save("fgz_dynamics_checkpoint.pth")

    if lr_scheduler is not None:
        lr_scheduler.step()

In [ ]:
# trainer.save("fgz_dynamics_trained.pth")

In [ ]:
# trainer = FGZTrainer.load("fgz_dynamics_trained.pth", agent)

In [ ]:
trainer.evaluate("MineRLBasaltMakeWaterfall-v0", render=True, max_steps=256, force_no_escape=True)